In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler 

In [ ]:
dados = pd.read_table("../datasets/Potencial_Novos_Clientes.txt",
                      sep="\t",
                      header=0)
dados.head()

#### Transformer

In [ ]:
# Podemos criar nossos próprios estimadores e transformers
class ColumnSelector(BaseEstimator, TransformerMixin):
    '''Seleciona um subset de um dado dataframe a partir de uma lista de colunas'''
    def __init__(self, cols_list):
        self.cols_list = cols_list
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, self.cols_list]

In [ ]:
cols_list_quanti = ['IDADE_CLIENTE',
                  'RENDA_MENSAL_CLIENTE',
                  'BEHAVIOUR_SCORE_CLIENTE',
                  'QTD_TRANSACOES_3M',
                  'QTD_ITENS_3M',
                  'VALOR_GASTO_3M',
                  'TICKET_MEDIO_3M']

dados_subset = ColumnSelector(cols_list_quanti)
dados_subset.transform(dados).head()


#### Pipeline

In [ ]:
num_pipe = Pipeline([('get_num_cols', ColumnSelector(cols_list_quanti)),
                ('fix_nan', SimpleImputer(missing_values=np.nan, strategy='median')),
                ('scale_data', MinMaxScaler())
])

In [ ]:
dados_transformados = pd.DataFrame(num_pipe.fit_transform(dados))
dados_transformados.head()

#### ColumnTransformer

In [ ]:
cols_list_quali= ['FLAG_ELETRONICOS_3M',
                 'SATISFACAO_ULTIMA_COMPRA']

In [ ]:
FeatureEng = ColumnTransformer(
    transformers=[
        ('cat_ohe', OneHotEncoder(), cols_list_quali),
        ('num_pipe', num_pipe, cols_list_quanti)
    ]
)
pd.DataFrame(FeatureEng.fit_transform(dados))

#### Como aplicar

In [ ]:
from sklearn.model_selection import RandomizedSearchCV                                                
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from scipy.stats import loguniform 
import warnings
warnings.filterwarnings('ignore')


In [ ]:
lista_X_quanti = ['IDADE_CLIENTE',
                  'RENDA_MENSAL_CLIENTE',
                  'BEHAVIOUR_SCORE_CLIENTE',
                  'QTD_TRANSACOES_3M',
                  'QTD_ITENS_3M',
                  'VALOR_GASTO_3M',
                  'TICKET_MEDIO_3M']

lista_X_quali = ['FLAG_ELETRONICOS_3M',
                 'SATISFACAO_ULTIMA_COMPRA']

In [ ]:
y = dados['VALOR_GASTO_PROX_12M']
X = dados[lista_X_quanti + lista_X_quali]

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=123)                                                        

In [ ]:
#X_teste.to_csv("X_teste.csv", index=False)

In [ ]:
class Wrapper(BaseEstimator):
    def __init__(self, intermediate_model):                
        self.intermediate_model = intermediate_model
    def fit(self, X, y=None):                        
        return self                                  
    def transform(self, X_teste):
        return self.intermediate_model.predict(X_teste)

In [ ]:
preprocessador = ColumnTransformer(transformers=[
    ("quanti", StandardScaler(), lista_X_quanti),
    ("quali", OneHotEncoder(
        sparse_output=False,drop="first", handle_unknown='ignore'),lista_X_quali)
])

In [ ]:
RL = Pipeline([
        ("preprocess", preprocessador),
        ("linear regression", RandomizedSearchCV(estimator=LinearRegression(),
                  param_distributions={},
                  scoring='neg_root_mean_squared_log_error',
                  cv=10) )
])

In [ ]:
RD =  Pipeline([
        ("preprocess", preprocessador),
        ("linear regression", RandomizedSearchCV(estimator=Ridge(),
                  param_distributions={'alpha': loguniform(1e-5, 1e1) },
                  scoring='neg_root_mean_squared_log_error',
                  cv=10) )
])

In [ ]:
treino_predict_pipe = FeatureUnion([
    ('linear', Wrapper(RL.fit(X_treino, y_treino))),
    ('ridge', Wrapper(RD.fit(X_treino, y_treino)))
])

In [ ]:
y_hat = treino_predict_pipe.fit_transform(X_teste)
y_hat[:10]

In [ ]:
model_select = int(len(y_hat)/2)

In [ ]:
from sklearn.metrics import root_mean_squared_error
print(root_mean_squared_error(y_teste, y_hat[:model_select]))
print(root_mean_squared_error(y_teste, y_hat[model_select:]))